In [1]:
dbutils.widgets.text("backup_filename", "", "")
dbutils.widgets.get("backup_filename")
backup_filename = getArgument("backup_filename")

dbutils.widgets.text("backup_folder", "", "")
dbutils.widgets.get("backup_folder")
backup_folder = getArgument("backup_folder")

dbutils.widgets.text("table", "", "")
dbutils.widgets.get("table")
table = getArgument("table")

print ("Param -\'input':")
print (backup_filename)
spark.conf.set("dfs.adls.oauth2.access.token.provider.type", "ClientCredential")
spark.conf.set("dfs.adls.oauth2.client.id", "f55ddaf6-8ebf-4474-aa00-4b06d8479c61")
spark.conf.set("dfs.adls.oauth2.credential", "9KlkgWv9WMqoDatfLINVLuJMsa057qLob+bdOj1aEv4=")
spark.conf.set("dfs.adls.oauth2.refresh.url", "https://login.microsoftonline.com/ae5ab930-6b7b-4b0c-9a92-dcc7e8008cb9/oauth2/token ")

Param -'input':
2018-08-30_00_53_44_noquotes.csv

In [2]:
import time

if backup_filename == "LATEST" or backup_filename == "":
  date_format = "%Y-%m-%d_%H_%M_%S"

  backup_files = [time.strptime(filename.name, date_format + ".csv") for filename in dbutils.fs.ls("adl://linxccdls02.azuredatalakestore.net/Backup/" + backup_folder)]
  backup_filename = time.strftime(date_format + ".csv", max(backup_files))
  print("Most recent file: " + backup_filename)
else:
  print('Chosen backup: {0}'.format(backup_filename))


Chosen backup: 2018-08-30_00_53_44_noquotes.csv

In [3]:
infile = "/Backup/" + backup_folder + "/" + backup_filename

print(infile)

myRecords = spark.read.option("delimiter", ",").csv("adl://linxccdls02.azuredatalakestore.net/" + infile, quote='"', escape='"', header=True)
myRecords.createOrReplaceTempView("myRecords")

/Backup/PAY34T/2018-08-30_00_53_44_noquotes.csv

In [4]:
sdf = spark.sql("""
SELECT 
       *
   FROM myRecords AS m
""")
sdf.createOrReplaceTempView("sdf")

In [5]:
spark.sql("DROP TABLE IF EXISTS " + table + "_temp")

spark.sql("""CREATE TABLE """ + table + """_temp
  USING CSV
    AS
    SELECT *
      FROM sdf
          """)

Out[ 96 ]: DataFrame[]

In [6]:
spark.sql("DROP TABLE IF EXISTS " + table)

spark.sql("ALTER TABLE " + table + "_temp RENAME TO " + table)

Out[ 97 ]: DataFrame[]